In [1]:
import random
import os
import re
import json
import numpy as np
import pandas as pd
# import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# import lightgbm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
import torch
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from transformers import ViltProcessor, ViltModel
# from transformers import ClipProcessor, ClipModel
import clip

from PIL import Image

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-10 10:50:57.313843: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 10:50:58.994539: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-10 10:50:58.994645: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
## 获取文本特征，以“title_subcatory_catogory_concept_alltags”的方式进行拼接
all_data = pd.read_csv('../data/combine_data_530.csv', low_memory=False, index_col=0)
all_data = all_data.fillna('0')
all_data

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,ispro,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,1,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,1,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,1,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,0,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,1,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1


In [4]:
all_data['all_text'] = all_data['Title'] + all_data['Subcategory'] + all_data['Concept'] + all_data['Category'] + all_data['Alltags']
all_data

,Category,Concept,Pid,Uid,Subcategory,Mediastatus,Pathalias,Ispublic,Alltags,Mediatype,...,user_description,canbuypro,timezone_timezone_id,photo_firstdate,timezone_offset,img,img_file,label,train_type,all_text
0,Fashion,glam,775,59@N75,Fashion,ready,luisdrayton,1,rock punk transgender tranny electronicmusic e...,photo,...,"0.0866962,-0.0752717,-0.0149407,-0.0460858,-0....",0,9,1213743830,1,train/59@N75/775.jpg,train/59@N75/775.jpg,11.18,0,Luis Drayton - Edinburgh shoot #6FashionglamFa...
1,Travel&Active&Sports,fifa,1075,1@N18,Soccer,ready,claudiolara,1,brazil rio brasil riodejaneiro by maria fifa m...,photo,...,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1@N18/1075.jpg,train/1@N18/1075.jpg,15.15,0,Arena da Barra - Arena HSBC - Arena do PAN #...
2,Entertainment,cinema,4890,351@N64,Movies,ready,rizzato,1,old cinema beauty marilyn photoshop movie joke...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,11,1296417600,-3,train/351@N64/4890.jpg,train/351@N64/4890.jpg,10.99,0,MARILYN 2015MoviescinemaEntertainmentold cinem...
3,Holiday&Celebrations,old,6568,6@N59,Birthday,ready,4kleuren,1,pictures old family scans brothers sister 1958...,photo,...,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/6@N59/6568.jpg,train/6@N59/6568.jpg,8.63,0,Knikkertijd - 1959BirthdayoldHoliday&Celebrati...
4,Food,thirsty,7079,1617@N40,Drinks,ready,glosackmd,1,hot sahara animal animals desert bottles drink...,photo,...,"0.14628,-0.0255633,-0.0711352,0.120531,-0.0728...",0,2,1263266743,0,train/1617@N40/7079.jpg,train/1617@N40/7079.jpg,11.16,0,CAMELS01DrinksthirstyFoodhot sahara animal ani...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,Travel&Active&Sports,sea,1118324,49686@N77,Seattle,ready,larsgerrits,1,natur nature frog sea lake fish outdoor macro ...,photo,...,"0.150263,0.00344591,-0.158953,0.118596,-0.0566...",0,8,1394046082,0,test/49686@N77/1118324.jpg,test/49686@N77/1118324.jpg,-1.00,-1,Quaak!SeattleseaTravel&Active&Sportsnatur natu...
486190,Travel&Active&Sports,cars,1118329,31147@N75,Cars,ready,None,1,11 arisk okruh veternov 2016 10 jn preov veter...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1460319331,0,test/31147@N75/1118329.jpg,test/31147@N75/1118329.jpg,-1.00,-1,arisk okruh veternov 2016CarscarsTravel&Active...
486191,Animal,cat,1118336,5656@N26,Cats,ready,rogersg,1,holland netherlands amsterdam cat nl ramses gu...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,9,1464117797,1,test/5656@N26/1118336.jpg,test/5656@N26/1118336.jpg,-1.00,-1,Bike & Cat / Eerste Anjeliers DwarsstraatCatsc...
486192,Whether&Season,white,1118349,28302@N36,Snowing,ready,ripensis,1,street people bw white black netherlands monoc...,photo,...,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",0,2,1246915883,0,test/28302@N36/1118349.jpg,test/28302@N36/1118349.jpg,-1.00,-1,The Stairs crewSnowingwhiteWhether&Seasonstree...


In [7]:
## 获取BERT的文本特征
# all_data['all_text']
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import trange
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("../model/bertweet_base", padding = True, truncation = True, use_fast=False, return_tensors="pt")
model = AutoModel.from_pretrained("../model/bertweet_base").to(device)
with torch.no_grad():
    bttext1 = torch.tensor(tokenizer.encode(all_data['all_text'][0])).to(device)
    # input_ids = torch.tensor([tokenizer.encode(line)])
    bttokenizer1 = tokenizer(bttext1, padding = True, truncation = True, return_tensors="pt").to(device)
    btoutputs1 = model(**bttokenizer1)
    btbert_fea = btoutputs1.pooler_output
    for index in trange(486193):
        bttext = all_data['all_text'][index + 1]
        btinputs = tokenizer(bttext, padding = True, truncation = True, return_tensors="pt").to(device)
        btoutputs = model(**btinputs)
        btbert_fea1 = btoutputs.pooler_output
        btbert_fea = torch.concat((btbert_fea,btbert_fea1),0)
btbert_fea_768 = pd.DataFrame(btbert_fea.cpu().detach().numpy())
btbert_fea_768.columns = ['bertweet_fea_{}'.format(i) for i in range(768)]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../model/bertweet_base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.